Reference: https://pyez.readthedocs.io/en/latest/index.html

In [6]:
from jnpr.junos import Device

with Device(host='10.85.165.185',user='labroot',password='lab123') as dev:
    print(dev.facts)

{'2RE': False, 'HOME': '/var/home/labroot', 'RE0': {'mastership_state': 'master', 'status': 'OK', 'model': 'RE-PTX1000', 'last_reboot_reason': '0x1:power cycle/failure', 'up_time': '8 days, 19 hours, 29 minutes, 25 seconds'}, 'RE1': None, 'RE_hw_mi': False, 'current_re': ['master', 'node', 'fwdd', 'member', 'pfem', 're0', 'localre'], 'domain': 'ultralab.juniper.net', 'fqdn': 'h19-28.ultralab.juniper.net', 'hostname': 'h19-28', 'hostname_info': {'re0': 'h19-28'}, 'ifd_style': 'CLASSIC', 'junos_info': {'re0': {'text': '18.2X75-D61.8', 'object': junos.version_info(major=(18, 2), type=X, minor=(75, 'D', 61), build=8)}}, 'master': 'RE0', 'model': 'PTX1000', 'model_info': {'re0': 'PTX1000'}, 'personality': 'PTX', 're_info': {'default': {'0': {'mastership_state': 'master', 'status': 'OK', 'model': 'RE-PTX1000', 'last_reboot_reason': '0x1:power cycle/failure'}, 'default': {'mastership_state': 'master', 'status': 'OK', 'model': 'RE-PTX1000', 'last_reboot_reason': '0x1:power cycle/failure'}}}, '

# Mapping cli command with RPC method

PyEZ supports low level capabilities to allow user to invoke XML RPCs in Python

Each cli command could be mapped with a XML RPC.

There are multiple ways to get the RPC based on the cli command.

1.From Junos cli, adding "| display xml rpc" after each cli command

labroot@grapevine-re0> show version | display xml rpc

<rpc-reply xmlns:junos="http://xml.juniper.net/junos/18.2X75/junos">
    <rpc>
        <get-software-information>   
        </get-software-information>
    </rpc>
    <cli>
        <banner>[edit]</banner>
    </cli>
</rpc-reply>

2.use PyEZ function display_xml_rpc('


In [7]:
from jnpr.junos import Device
                                    
with Device(host='10.85.165.185',user='labroot',password='lab123') as dev:
    print(dev.display_xml_rpc('show version'))

<Element get-software-information at 0x254f3d30308>


# Pull Data from Junos Device using PYEZ

In [13]:
# We can interact with Juniper device directly with CLI command.

from jnpr.junos import Device

with Device(host='10.85.165.185', user='labroot', password='lab123') as dev:
    junos_version=dev.cli('show version')
    # lets see response of dev.cli(), it should be string 
    print(type(junos_version))
    print(junos_version)
    # using CLI command is not recommended so we will get warning to use RPC
    # to prevent the warning, use "warning=false" in the cli() method
    junos_version=dev.cli('show version', warning=False)
  

C:\Users\vlsingh\AppData\Roaming\Python\Python37\site-packages\jnpr\junos\device.py:724: RuntimeWarning: 
CLI command is for debug use only!
Instead of:
cli('show version')
Use:
rpc.get_software_information()

  warnings.warn(warning_string, RuntimeWarning)


<class 'str'>

Hostname: h19-28
Model: ptx1000
Junos: 18.2X75-D61.8
JUNOS OS Kernel 64-bit  [20200423.d6acdc5_builder_stable_11]
JUNOS OS libs [20200423.d6acdc5_builder_stable_11]
JUNOS OS runtime [20200423.d6acdc5_builder_stable_11]
JUNOS OS time zone information [20200423.d6acdc5_builder_stable_11]
JUNOS network stack and utilities [20200919.064606_builder_junos_182_x75_d61]
JUNOS libs [20200919.064606_builder_junos_182_x75_d61]
JUNOS OS libs compat32 [20200423.d6acdc5_builder_stable_11]
JUNOS OS 32-bit compatibility [20200423.d6acdc5_builder_stable_11]
JUNOS libs compat32 [20200919.064606_builder_junos_182_x75_d61]
JUNOS runtime [20200919.064606_builder_junos_182_x75_d61]
Junos vmguest package [20200919.064606_builder_junos_182_x75_d61]
JUNOS sflow ptx [20200919.064606_builder_junos_182_x75_d61]
JUNOS py extensions [20200919.064606_builder_junos_182_x75_d61]
JUNOS py base [20200919.064606_builder_junos_182_x75_d61]
JUNOS OS vmguest [20200423.d6acdc5_builder_stable_11]
JUNOS OS crypt

In [23]:
# Another method is to import the data from shell
# import StartShell class to interact with shell directly 
# All the PFE and Linux commands could be collected in this method 

from jnpr.junos import Device
from jnpr.junos.utils.start_shell import StartShell
from pprint import pprint

with Device(host='10.85.165.185',user='labroot',password='lab123') as dev:
    ss = StartShell(dev)
    ss.open()
    details=ss.run('uname -a')
    fpc=ss.run('cli -c "show chassis fpc"')
    ss.close()
    
# Response is a tuple

print(type(details))
pprint(details)
pprint(fpc)
    

<class 'tuple'>
(True,
 'uname -a\r'
 '\r\n'
 'FreeBSD h19-28 JNPR-11.0-20200423.d6acdc5_buil FreeBSD '
 'JNPR-11.0-20200423.d6acdc5_builder_stable_11 #0 r356482+d6acdc5c9b8(HEAD): '
 'Thu Apr 23 13:37:51 PDT 2020     '
 'builder@feyrith.juniper.net:/volume/build/junos/occam/llvm-3.7/sandbox-20200416/freebsd/stable_11/20200416.020549_builder_stable_11.d6acdc5/obj/amd64/juniper/kernels/JNPR-AMD64-PRD/kernel  '
 'amd64\r\n'
 '% ')
(True,
 'cli -c "show chassis fpc"\r'
 '\r\n'
 '                     Temp  CPU Utilization (%)   CPU Utilization (%)  '
 'Memory    Utilization (%)\r\n'
 'Slot State            (C)  Total  Interrupt      1min   5min   15min  DRAM '
 '(MB) Heap     Buffer\r\n'
 '  0  Online            24     23          0       23     23     23    '
 '32768      98         84\r\n'
 '% ')


# Interacting with the router using XML RPC

In [12]:
from jnpr.junos import Device
from jnpr.junos.utils.start_shell import StartShell
from lxml import etree
from pprint import pprint

#etree is needed to iterate over xml output

with Device(host='10.85.160.181', user='labroot', password='lab123') as dev:
    # Make a RPC call using dev.rpc
    rpc_fpc=dev.display_xml_rpc('show chassis fpc')
    print(rpc_fpc)
    fpc_xml=dev.rpc.get_fpc_information()
    # response is a lxml.etree element
    # need to import etree from lxml to interpret the output
    
    print(type(fpc_xml))
    pprint(etree.dump(fpc_xml))
    
    # There are many ways to iterate over the XML content 
    
    #1) findall (): find all matching etree blocks based on input xpath
    #2) findtext (): return the content based on the input xpath
    
    for fpc in fpc_xml.findall('./fpc'):
        slot=fpc.findtext('slot')
        cpu_15min=fpc.findtext('cpu-15min-avg')
        print(f'Slot: {slot} , cpu-15min-avg: {cpu_15min} ')
    
    
    #the response data type could be converted into json text or json format:
    
    #exaple text format:
    
    fpc_text=dev.rpc.get_fpc_information({'format':'text'})
    
    #RPC response is still a lxml.etree element, but the content is in plain text instead of XML format
    
    print(type(fpc_text))
    print(etree.dump(fpc_text))
    
    #example json format:
    
    # In JSON format the RPC response is a python native dict.
    fpc_json=dev.rpc.get_fpc_information({'format':'json'})
    print(type(fpc_json))
    pprint(fpc_json)

<Element get-fpc-information at 0x2c3628c76c8>
<class 'lxml.etree._Element'>
<fpc-information style="brief">
  <fpc>
    <slot>0</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>1</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>2</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>3</slot>
    <state>Empty</state>
  </fpc>
  <fpc>
    <slot>4</slot>
    <state>Online</state>
    <temperature celsius="42">42</temperature>
    <cpu-total>8</cpu-total>
    <cpu-interrupt>0</cpu-interrupt>
    <cpu-1min-avg>8</cpu-1min-avg>
    <cpu-5min-avg>8</cpu-5min-avg>
    <cpu-15min-avg>8</cpu-15min-avg>
    <memory-dram-size>3584</memory-dram-size>
    <memory-heap-utilization>6</memory-heap-utilization>
    <memory-buffer-utilization>13</memory-buffer-utilization>
  </fpc>
  <fpc>
    <slot>5</slot>
    <state>Online</state>
    <temperature celsius="45">45</temperature>
    <cpu-total>9</cpu-total>
    <cpu-interrupt>0</cpu-interrupt>
    <cpu-1min-avg>9</cpu-1min-avg>

# PyEZ table and views

Instead of iterating the RPC response, PyEZ offers "Table and Views" feature to invoke RPC and map the response into a Python data structure. Tables and Views are YAML file which contains the desired RPC method and specific fields for matching.

There are pre-defined Tables and we could build our own.

The Junos PyEZ jnpr.junos.op module provides predefined Table and View definitions for RPCs corresponding to some common operational commands

https://www.juniper.net/documentation/en_US/junos-pyez/topics/reference/general/junos-pyez-tables-op-predefined.html

In [13]:
# Example use one of the pre-exisiting table (FPcInfoTable)

from jnpr.junos import Device

from jnpr.junos.op.fpc import FpcInfoTable
from pprint import pprint

# jnpr.junos.op module provides predefined tables and views

# FpcInfoTable is same as RPC (get-fpc-information) and cli (show chassis fpc)

with Device(host='10.85.160.181', user='labroot', password='lab123') as dev:
    
    # FpcInfoTable object is initiated
    fpc_table=FpcInfoTable(dev)
    
    #use get to get the output
    
    fpc_table.get()
    
    print(type(fpc_table))
    
    print(fpc_table)
    
    print(fpc_table.items())
    
    pprint(fpc_table.items())

<class 'jnpr.junos.factory.OpTable.FpcInfoTable'>
FpcInfoTable:10.85.160.181: 6 items
[('0', [('state', 'Empty'), ('memory', None), ('cpu', None)]), ('1', [('state', 'Empty'), ('memory', None), ('cpu', None)]), ('2', [('state', 'Empty'), ('memory', None), ('cpu', None)]), ('3', [('state', 'Empty'), ('memory', None), ('cpu', None)]), ('4', [('state', 'Online'), ('memory', '6'), ('cpu', '10')]), ('5', [('state', 'Online'), ('memory', '6'), ('cpu', '8')])]
[('0', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('1', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('2', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('3', [('state', 'Empty'), ('memory', None), ('cpu', None)]),
 ('4', [('state', 'Online'), ('memory', '6'), ('cpu', '10')]),
 ('5', [('state', 'Online'), ('memory', '6'), ('cpu', '8')])]
